In [2]:
import torch
from trl import SFTTrainer
import os
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
import time

from src.Basic_LLM_functions import LLM_conection
from src.Dataset_generation import generate_full_question,save_question_and_log,subtopic_prompt_template,question_prompt_template
from src.Chain_of_thought import Chain_of_thought

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="LLMs/Meta-Llama-3.1-8B",#-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 24.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LLMs/Meta-Llama-3.1-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [3]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

model = FastLanguageModel.for_inference(model)

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.
/home/jaime/miniconda3/envs/llama_finetuning_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
LLM_conection.set_model(model,tokenizer)

In [5]:
num_questions = 99  # Change this to the number of questions you want to generate

for i in range(num_questions):
    print(f"Generating question {i+1}/{num_questions}")
    question_data = generate_full_question(model,tokenizer)
    save_question_and_log(question_data, subtopic_prompt_template, question_prompt_template)
    print(f"Main Topic: {question_data['main_topic']}")
    print(f"Subtopic: {question_data['subtopic']}")
    print(f"Question: {question_data['question']}")
    print("\n---\n")
    time.sleep(2)  # Add a delay to avoid hitting API rate limits

print("Question generation complete. Check the 'questions' and 'log' folders for output.")

Generating question 1/99
Main Topic: Political Science
Subtopic: Unpacking the implications of cosmopolitanism on transnational governance in the Anthropocene era.
Question: What are the implications of cosmopolitanism's ontological and epistemological commitments on its capacity to facilitate transnational cooperation in the face of climate change, considering the tensions between universalistic and particularistic perspectives on justice, human rights, and the Anthropocene era's unique challenges to traditional notions of sovereignty and territoriality, and how might this inquiry be reframed through the lens of post-structuralist and post-colonial critiques of grand narratives and Enlightenment rationality, which could, in turn, inform the development of a more nuanced and reflexive cosmopolitanism that acknowledges and engages with the complexities of difference and power in the contemporary global order?

---

Generating question 2/99
Main Topic: Economics
Subtopic: Adapting the me

KeyboardInterrupt: 

In [3]:
os.makedirs("answers", exist_ok=True)
answered_questions=os.listdir("answers")

In [4]:
txt_files_contents = []
folder_path="questions"
for filename in os.listdir(folder_path):
    if filename in answered_questions:
        continue
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), 'r') as file:
            txt_files_contents.append((filename,file.read()))

In [9]:
dataset=[]
for filename,question in txt_files_contents:
    output=Chain_of_thought(question)
    
    if len(output["reasoning"])>0:
        #Save the json to .json in a folder called answers
        with open(f"answers/{filename}.json", 'w') as f:
            json.dump(output, f, indent=4)
    
    break#We only wanna do 1 chain of thought for now

----------------------
Attempt 1 for question: What are the key methodological adaptations required to apply the phenomenological analysis framework to the cultural significance of contemporary street art, and how can these adaptations be evaluated in terms of their effectiveness in uncovering the underlying themes and meanings that street artists intend to convey through their work?
----------------------
Additional information:
To apply the phenomenological analysis framework to the cultural significance of contemporary street art, several key methodological adaptations are required. These adaptations involve a nuanced understanding of the research context, the street art phenomenon, and the phenomenological approach.

1.  **Contextualizing Street Art**: Phenomenological analysis requires an in-depth understanding of the context in which street art is produced and consumed. This includes the social, political, and economic factors that influence the creation and reception of street a

NameError: name 'json' is not defined